# Pipeline

1. **Preprocessing**
2. Data Extraction
3. Data Exploration
4. Model

This file preprocesses the data. It handles missing values, assigns correct datatypes and generates a .pkl file that contains a results dataframe that can be used for the second step.

# Imports

In [7]:
import pandas as pd
import numpy as np
from collections import defaultdict

# Data preprocessing

In [8]:
results = pd.read_csv('csv_data/results.csv')

In [9]:
results.head(10)

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,E0,14/08/10,Aston Villa,West Ham,3.0,0.0,H,2.0,0.0,H,...,11.0,2.0,15.0,15.0,16.0,7.0,1.0,2.0,0.0,0.0
1,E0,14/08/10,Blackburn,Everton,1.0,0.0,H,1.0,0.0,H,...,2.0,12.0,19.0,14.0,1.0,3.0,2.0,1.0,0.0,0.0
2,E0,14/08/10,Bolton,Fulham,0.0,0.0,D,0.0,0.0,D,...,9.0,7.0,12.0,13.0,4.0,8.0,1.0,3.0,0.0,0.0
3,E0,14/08/10,Chelsea,West Brom,6.0,0.0,H,2.0,0.0,H,...,13.0,4.0,10.0,10.0,3.0,1.0,1.0,0.0,0.0,0.0
4,E0,14/08/10,Sunderland,Birmingham,2.0,2.0,D,1.0,0.0,H,...,2.0,7.0,13.0,10.0,3.0,6.0,3.0,3.0,1.0,0.0
5,E0,14/08/10,Tottenham,Man City,0.0,0.0,D,0.0,0.0,D,...,18.0,7.0,13.0,16.0,10.0,3.0,0.0,2.0,0.0,0.0
6,E0,14/08/10,Wigan,Blackpool,0.0,4.0,A,0.0,3.0,A,...,6.0,7.0,8.0,11.0,6.0,4.0,1.0,1.0,0.0,0.0
7,E0,14/08/10,Wolves,Stoke,2.0,1.0,H,2.0,0.0,H,...,7.0,6.0,17.0,13.0,5.0,5.0,0.0,2.0,0.0,0.0
8,E0,15/08/10,Liverpool,Arsenal,1.0,1.0,D,0.0,0.0,D,...,4.0,7.0,13.0,15.0,9.0,11.0,1.0,3.0,1.0,1.0
9,E0,16/08/10,Man United,Newcastle,3.0,0.0,H,2.0,0.0,H,...,10.0,3.0,9.0,5.0,5.0,3.0,2.0,2.0,0.0,0.0


In [10]:
results.loc[results['Div'] == 'E0', 'Div'] = 'E1'    # first English division should be called 'E1' instead of 'E0'

## Handling missing values

In [11]:
results[results['HY'].isnull()]

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6969,I1,23/09/12,Cagliari,Roma,0.0,3.0,A,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7945,I1,02/03/15,Roma,Juventus,1.0,1.0,D,0.0,0.0,D,...,3.0,1.0,11.0,16.0,1.0,5.0,NaN,5.0,1.0,0.0
8078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
results = results.dropna(axis='index', how='all')  # drop all rows that only contain NaN values

In [13]:
results.loc[7945, 'HY'] = 6    # add the missing value by hand

In [14]:
results.drop(labels=6969, axis='index', inplace=True)  # remove row that misses too many values

In [15]:
results[results['HTHG'].isnull()]

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
8477,I1,28/08/16,Sassuolo,Pescara,0.0,3.0,A,NaN,NaN,NaN,...,3.0,5.0,12.0,21.0,2.0,7.0,2.0,2.0,0.0,0.0


In [16]:
results.drop(labels=8477, axis='index', inplace=True)  # remove game with non-representative result

In [17]:
# all missing values have been dealt with by this point
for col in results.columns:
    print(col, len(results[results[col].isnull()]))

Div 0
Date 0
HomeTeam 0
AwayTeam 0
FTHG 0
FTAG 0
FTR 0
HTHG 0
HTAG 0
HTR 0
HS 0
AS 0
HST 0
AST 0
HF 0
AF 0
HC 0
AC 0
HY 0
AY 0
HR 0
AR 0


## Datatypes

In [18]:
# convert float columns to int
for col in results.columns:
    if results[col].dtype == np.float64:
        results[col] = results[col].astype(int)

In [19]:
results.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,E1,14/08/10,Aston Villa,West Ham,3,0,H,2,0,H,...,11,2,15,15,16,7,1,2,0,0
1,E1,14/08/10,Blackburn,Everton,1,0,H,1,0,H,...,2,12,19,14,1,3,2,1,0,0
2,E1,14/08/10,Bolton,Fulham,0,0,D,0,0,D,...,9,7,12,13,4,8,1,3,0,0
3,E1,14/08/10,Chelsea,West Brom,6,0,H,2,0,H,...,13,4,10,10,3,1,1,0,0,0
4,E1,14/08/10,Sunderland,Birmingham,2,2,D,1,0,H,...,2,7,13,10,3,6,3,3,1,0


In [20]:
# Convert Date column to proper datetime format.
# It remains an absolute mistery why this has to be done twice, but DON'T CHANGE IT!
#results['Date'] = pd.to_datetime(results['Date']).dt.strftime('%d-%m-%Y')
#results['Date'] = pd.to_datetime(results['Date']).dt.strftime('%d-%m-%Y')

results['Date'] = pd.to_datetime(results['Date'], dayfirst=True).dt.date

In [21]:
results[9470:9485]

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
9478,I1,2019-03-03,Udinese,Bologna,2,1,H,1,1,D,...,6,9,21,20,6,6,3,3,0,0
9479,I1,2019-03-08,Juventus,Udinese,4,1,H,2,0,H,...,6,2,7,9,5,0,0,3,0,0
9480,I1,2019-03-09,Chievo,Milan,1,2,A,1,1,D,...,1,3,20,13,4,1,3,1,0,0
9481,I1,2019-03-09,Parma,Genoa,1,0,H,0,0,D,...,4,1,12,12,4,5,2,1,0,0
9482,I1,2019-03-10,Bologna,Cagliari,2,0,H,1,0,H,...,8,6,16,21,8,3,0,0,0,0
9483,I1,2019-03-10,Fiorentina,Lazio,1,1,D,0,1,A,...,1,6,8,14,4,7,3,1,0,0
9484,I1,2019-03-10,Frosinone,Torino,1,2,A,1,0,H,...,5,7,14,21,3,3,2,4,0,0
9485,I1,2019-03-10,Inter,Spal,2,0,H,0,0,D,...,4,1,17,17,4,7,3,5,0,0
9486,I1,2019-03-10,Sampdoria,Atalanta,1,2,A,0,0,D,...,6,6,13,10,4,7,2,2,0,0
9487,I1,2019-03-10,Sassuolo,Napoli,1,1,D,0,0,D,...,1,4,11,10,3,5,3,2,0,0


In [22]:
results[11792:11797]

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
11800,SP1,2016-04-01,Vallecano,Getafe,2,0,H,1,0,H,...,7,3,10,10,9,6,4,5,1,0
11801,SP1,2016-04-02,Ath Madrid,Betis,5,1,H,2,0,H,...,12,5,14,12,5,5,1,3,0,0
11802,SP1,2016-04-02,Barcelona,Real Madrid,1,2,A,0,0,D,...,3,6,17,15,4,3,3,2,0,1
11803,SP1,2016-04-02,Celta,La Coruna,1,1,D,1,1,D,...,2,3,15,8,11,0,4,2,0,1
11804,SP1,2016-04-02,Las Palmas,Valencia,2,1,H,0,1,A,...,6,4,8,17,8,2,2,4,0,0


In [23]:
results.dtypes

Div         object
Date        object
HomeTeam    object
AwayTeam    object
FTHG         int32
FTAG         int32
FTR         object
HTHG         int32
HTAG         int32
HTR         object
HS           int32
AS           int32
HST          int32
AST          int32
HF           int32
AF           int32
HC           int32
AC           int32
HY           int32
AY           int32
HR           int32
AR           int32
dtype: object

In [24]:
# Assign an ID to each team for easier identification
team_ids = defaultdict()

assert len(results['HomeTeam'].unique()) == len(results['AwayTeam'].unique())

teams = results['HomeTeam'].unique()

index = 0
for team in teams:
    team_ids[team] = index
    print(index, team)    
    index += 1

0 Aston Villa
1 Blackburn
2 Bolton
3 Chelsea
4 Sunderland
5 Tottenham
6 Wigan
7 Wolves
8 Liverpool
9 Man United
10 Arsenal
11 Birmingham
12 Everton
13 Stoke
14 West Brom
15 West Ham
16 Fulham
17 Newcastle
18 Man City
19 Blackpool
20 QPR
21 Swansea
22 Norwich
23 Reading
24 Southampton
25 Crystal Palace
26 Hull
27 Cardiff
28 Leicester
29 Burnley
30 Bournemouth
31 Watford
32 Middlesbrough
33 Brighton
34 Huddersfield
35 Bayern Munich
36 FC Koln
37 Freiburg
38 Hamburg
39 Hannover
40 Hoffenheim
41 M'gladbach
42 Dortmund
43 Mainz
44 Kaiserslautern
45 Ein Frankfurt
46 Nurnberg
47 Schalke 04
48 St Pauli
49 Werder Bremen
50 Wolfsburg
51 Leverkusen
52 Stuttgart
53 Augsburg
54 Hertha
55 Greuther Furth
56 Fortuna Dusseldorf
57 Braunschweig
58 Paderborn
59 Darmstadt
60 Ingolstadt
61 RB Leipzig
62 Roma
63 Udinese
64 Bari
65 Chievo
66 Fiorentina
67 Milan
68 Palermo
69 Parma
70 Sampdoria
71 Bologna
72 Cagliari
73 Cesena
74 Inter
75 Brescia
76 Catania
77 Genoa
78 Juventus
79 Lazio
80 Lecce
81 Napoli
82 

In [25]:
# Convert team name columns to IDs
#results['HomeID'] = results['HomeTeam']
#results['AwayID'] = results['AwayTeam']
results.rename(columns={'HomeTeam': 'HomeID', 'AwayTeam': 'AwayID'}, inplace=True)
results['HomeID'].replace(team_ids, inplace=True)
results['AwayID'].replace(team_ids, inplace=True)

In [26]:
# Replace H, D and A by 1, 0 and 2 (target variables)
result_dict = {'H': 1, 'D': 0, 'A': 2}
results['FTR'].replace(result_dict, inplace=True)

In [28]:
results.index = range(len(results.index))
results = results.sort_index()

In [29]:
results[1890:1910]

,Div,Date,HomeID,AwayID,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
1890,E1,2015-05-24,10,14,4,1,1,4,0,H,...,13,5,6,7,7,3,1,0,0,0
1891,E1,2015-05-24,0,29,0,1,2,0,1,A,...,5,3,9,6,7,2,1,2,0,0
1892,E1,2015-05-24,3,4,3,1,1,1,1,D,...,8,9,12,8,11,3,2,1,0,0
1893,E1,2015-05-24,25,21,1,0,1,0,0,D,...,7,2,22,13,7,3,3,0,0,0
1894,E1,2015-05-24,12,5,0,1,2,0,1,A,...,1,3,12,8,3,5,1,2,0,0
1895,E1,2015-05-24,26,9,0,0,0,0,0,D,...,6,1,12,15,8,1,2,2,0,1
1896,E1,2015-05-24,28,20,5,1,1,2,0,H,...,7,2,7,6,5,6,0,0,0,0
1897,E1,2015-05-24,18,24,2,0,1,1,0,H,...,6,4,13,8,8,4,1,1,0,0
1898,E1,2015-05-24,17,15,2,0,1,0,0,D,...,4,1,9,9,2,3,2,1,0,0
1899,E1,2015-05-24,13,8,6,1,1,5,0,H,...,9,4,13,4,3,9,4,2,0,0


In [31]:
results.sample(6)

,Div,Date,HomeID,AwayID,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
428,E1,2011-09-18,5,8,4,0,1,1,0,H,...,16,2,10,9,4,1,1,4,0,2
12209,SP1,2017-04-27,106,117,2,1,1,0,0,D,...,9,3,10,17,11,1,1,3,0,0
2688,E1,2017-08-27,5,29,1,1,0,0,0,D,...,5,3,9,9,10,7,0,0,0,0
2375,E1,2016-10-29,31,26,1,0,1,0,0,D,...,0,2,13,8,7,2,1,2,0,0
10333,SP1,2012-05-05,105,98,2,1,1,0,1,A,...,2,4,20,14,8,5,6,2,0,0
3573,D1,2011-01-14,51,42,1,3,2,0,0,D,...,3,5,11,18,6,7,2,1,0,0


# Save dataframe as .pkl

In [30]:
results.to_pickle('preprocessed_results.pkl')